In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from logging import INFO, WARNING, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

# Import libraries

In [ ]:
import copy
import datetime
import gc
import glob
import os
import pathlib
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import yaml
from numpy.testing import assert_array_equal
from src.utils import read_pickle, set_seeds, write_pickle
from tqdm.notebook import tqdm

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Define constants

In [ ]:
ROOT_DIR = str((pathlib.Path(os.environ["PYTHONPATH"]) / "..").resolve())
DL_DATA_DIR = pathlib.Path(f"{ROOT_DIR}/data/DL_data")
DL_INFERENCE_DIR = pathlib.Path(f"{ROOT_DIR}/data/DL_inferences")
EXPERIMENT_NAME = "unet_model"

In [ ]:
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
CONFIG_PATHS = sorted(glob.glob(f"{ROOT_DIR}/pytorch/config/*.yml"))

In [ ]:
HR_BUILDING_HEIGHT_PATH = f"{ROOT_DIR}/datascience/script/EleTopoZ_HR.txt"
LR_BUILDING_HEIGHT_PATH = f"{ROOT_DIR}/datascience/script/EleTopoZ_LR.txt"

In [ ]:
CONFIGS = OrderedDict()

for config_path in CONFIG_PATHS:
    if "tutorial" in config_path:
        continue
    with open(config_path) as file:
        config = yaml.safe_load(file)

    config_name = os.path.basename(config_path).split(".")[0]
    assert config_name not in CONFIGS

    _dir = f"{ROOT_DIR}/data/DL_results/{EXPERIMENT_NAME}/{config_name}"

    CONFIGS[config_name] = {
        "config": config,
        "model_name": config["model"]["model_name"],
        "experiment_name": EXPERIMENT_NAME,
        "weight_path": f"{_dir}/weights.pth",
        "learning_history_path": f"{_dir}/learning_history.csv",
    }

In [ ]:
HEIGHT_LEVELS = np.arange(32) * 5 + 17.5  # meters
HEIGHT_LEVELS

# Define methods

In [ ]:
def read_building_height(
    building_path: str, target_col: str, margin: int = 0
) -> np.ndarray:

    with open(building_path, "r") as file:
        lines = file.readlines()

    cols = ["i", "j", "Ez", "Tz", "Tzl"]
    _dict = {}
    for i, line in enumerate(lines[1:]):  # skip header
        splits = list(
            map(lambda s: s.strip(), filter(lambda s: s != "", line.split(" ")))
        )
        _dict[i] = {k: v for k, v in zip(cols, splits)}

    df_topography = pd.DataFrame.from_dict(_dict).T

    for col in cols:
        if col == "i" or col == "j":
            df_topography[col] = df_topography[col].astype(int)
        else:
            df_topography[col] = df_topography[col].astype(float)

    ret = pd.pivot_table(
        data=df_topography[["i", "j", target_col]],
        values=target_col,
        index="i",
        columns="j",
        aggfunc="max",
    ).values

    if margin == 0:
        return ret
    else:
        return ret[margin:-margin, margin:-margin]


def infer_2m_height_temperature(tempearture: np.ndarray, building: np.ndarray):
    assert tempearture.ndim == 4  # batch, z, x, y dims
    assert building.ndim == 2  # x and y dims
    assert building.shape == tempearture.shape[2:]
    assert tempearture.shape[1] == len(HEIGHT_LEVELS)

    temperature_2m = np.full(
        shape=(tempearture.shape[0],) + building.shape,
        fill_value=np.nan,
        dtype=np.float32,
    )

    for i in tqdm(range(building.shape[0])):
        for j in range(building.shape[1]):
            b = building[i, j]

            if b > HEIGHT_LEVELS[-1]:
                continue

            # Get the first index above the building height
            first_id = 0
            if b > HEIGHT_LEVELS[0]:
                first_id = np.where(HEIGHT_LEVELS >= b)[0][0]
                assert HEIGHT_LEVELS[first_id - 1] < b <= HEIGHT_LEVELS[first_id]

            second_id = first_id + 1
            if second_id >= len(HEIGHT_LEVELS):
                t = tempearture[:, first_id, i, j]
            else:
                t1 = tempearture[:, first_id, i, j]
                t2 = tempearture[:, second_id, i, j]
                h1 = HEIGHT_LEVELS[first_id]
                h2 = HEIGHT_LEVELS[second_id]

                # linear extrapolation
                t = t1 + (b - h1) * (t2 - t1) / (h2 - h1)

            temperature_2m[:, i, j] = t
    return temperature_2m

# Read building height data

In [ ]:
# Drop sponge region on the lateral boundaries
hr_tz = read_building_height(HR_BUILDING_HEIGHT_PATH, "Tz", margin=40)

# Interpolate vertically

In [ ]:
for config_name in CONFIGS.keys():

    inference_dir = DL_INFERENCE_DIR / EXPERIMENT_NAME / config_name
    if not os.path.exists(inference_dir):
        logger.info(f"Inference dir does not exist. config = {config_name}")
        continue

    if os.path.exists(f"{DATA_DIR}/pred_2m_t_{config_name}.pickle"):
        logger.info(f"Result already exists. config = {config_name}")
        continue

    inference_paths = sorted(glob.glob(str(inference_dir / "*.npy")))
    assert len(inference_paths) == 900

    logger.info(f"\n{config_name} is being evaluated.")

    hr_dir_name = "10"
    hr_dir = DL_DATA_DIR / hr_dir_name

    pred_t = np.zeros(
        shape=(len(inference_paths), len(HEIGHT_LEVELS)) + hr_tz.shape, dtype=np.float32
    )
    true_t = np.zeros_like(pred_t)

    for idx, path in tqdm(enumerate(inference_paths), total=len(inference_paths)):
        pred = np.transpose(np.load(path).squeeze(), axes=(0, 1, 3, 2))
        # Axes = channel, height, east, north

        name = os.path.basename(path)
        name = name.replace("SR_x04", "HR")
        str_date = name.split("T")[0]
        hr_path = str(hr_dir / str_date / name)

        # Restrict height under 32 x 5 meters
        # 32 == len(HEIGHT_LEVELS)
        y = np.transpose(np.load(hr_path)[:, :32], axes=(0, 1, 3, 2))
        assert y.shape == pred.shape

        pred_t[idx] = pred[0]  # first channel is temperature
        true_t[idx] = y[0]

    pred_2m_t = infer_2m_height_temperature(pred_t, hr_tz)
    true_2m_t = infer_2m_height_temperature(true_t, hr_tz)

    write_pickle(pred_2m_t, f"{DATA_DIR}/pred_2m_t_{config_name}.pickle")
    write_pickle(true_2m_t, f"{DATA_DIR}/true_2m_t_{config_name}.pickle")

    pred_2m_t = read_pickle(f"{DATA_DIR}/pred_2m_t_{config_name}.pickle")
    true_2m_t = read_pickle(f"{DATA_DIR}/true_2m_t_{config_name}.pickle")

    logger.info(f"RMSE = {np.sqrt(np.nanmean((pred_2m_t - true_2m_t) ** 2))}")
    logger.info(f"MAE = {np.nanmean(np.abs(pred_2m_t - true_2m_t))}")